# モーター動作テスト

Modbus RTU 経由で MCU に速度指令を送り、前進・後退・停止の一連動作を確認します。

- ポート: `/dev/ttyAMA0` (GPIO UART)
- プロトコル: Modbus RTU (スレーブアドレス 1)
- レジスタ: 0x40 (左速度), 0x41 (右速度) [mm/s]
- 安全タイムアウト: 500ms 書き込み途絶で自動停止

In [ ]:
from pymodbus.client import ModbusSerialClient
import time

client = ModbusSerialClient(port="/dev/ttyAMA0", baudrate=115200, timeout=1)
client.connect()

result = client.read_holding_registers(0x00, count=2, device_id=1)
status = result.registers[0]
print(f"STATUS: IMU_OK={bool(status&1)}, Motor_OK={bool(status&2)}")

In [ ]:
SPEED = 100  # mm/s

def send_speed(left, right, duration):
    """速度指令を duration 秒間送り続ける (200ms間隔)"""
    def to_u16(v): return v if v >= 0 else v + 0x10000
    end = time.time() + duration
    while time.time() < end:
        client.write_registers(0x40, values=[to_u16(int(left)), to_u16(int(right))], device_id=1)
        time.sleep(0.2)

def stop():
    """即時停止"""
    client.write_registers(0x40, values=[0, 0], device_id=1)
    print("停止")

In [ ]:
print(f"前進 {SPEED} mm/s x 2秒")
send_speed(SPEED, SPEED, 2.0)
stop()
time.sleep(1.0)

print(f"後退 {SPEED} mm/s x 2秒")
send_speed(-SPEED, -SPEED, 2.0)
stop()
print("テスト完了")

In [ ]:
stop()
client.close()
print("切断完了")